# Explore here

In [2]:
import pandas as pd

# Lectura del conjunto de datos especificando el delimitador correcto
url = "https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv"
data = pd.read_csv(url, delimiter=';')  # Usar ';' como delimitador para corregir el problema de las columnas

# Imprimir los nombres de las columnas para verificar
print(data.columns)



Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')


División Conjunto Datos

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Codificar la variable objetivo 'y'
label_encoder = LabelEncoder()
data['y'] = label_encoder.fit_transform(data['y'])  # Transforma 'yes' en 1 y 'no' en 0

# Convertir variables categóricas en variables dummy
data = pd.get_dummies(data, drop_first=True)

# Dividir los datos en conjuntos de entrenamiento y prueba
X = data.drop('y', axis=1)
y = data['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Alinear X_train y X_test para que ambos tengan las mismas columnas
X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)  # Rellenar con ceros donde falten valores

Entrenamiento Modelo

In [5]:
from sklearn.linear_model import LogisticRegression

# Crear el modelo de regresión logistica
model = LogisticRegression(max_iter=10000, random_state=42, solver='saga')
model.fit(X_train, y_train)

print("Modelo entrenado con éxito.")


Modelo entrenado con éxito.


Predicción Modelo y Evaluación

In [6]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Realizar predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Precisión del modelo: {accuracy:.2f}")
print("Matriz de confusión:\n", conf_matrix)
print("Reporte de clasificación:\n", class_report)



Precisión del modelo: 0.91
Matriz de confusión:
 [[7125  178]
 [ 575  360]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.93      0.98      0.95      7303
           1       0.67      0.39      0.49       935

    accuracy                           0.91      8238
   macro avg       0.80      0.68      0.72      8238
weighted avg       0.90      0.91      0.90      8238



Optimización parametros Random Search

In [7]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Definir el espacio de hiperparámetros para RandomizedSearchCV
random_param_grid = {
    'C': np.logspace(-4, 4, 10),
    'penalty': ['l2'], 
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

# Crear el objeto RandomizedSearchCV
random_search = RandomizedSearchCV(model, random_param_grid, n_iter=10, cv=3, scoring='accuracy', random_state=42, verbose=1)

# Ajustar RandomizedSearchCV
random_search.fit(X_train, y_train)

# Mejores hiperparámetros y modelo
print("Mejores hiperparámetros:", random_search.best_params_)
best_random_model = random_search.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

Mejores hiperparámetros: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 2.782559402207126}


/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
from sklearn.metrics import accuracy_score

# Evaluar el modelo de Random Search
y_pred_random = best_random_model.predict(X_test)
print("Precisión del modelo de Random Search:", accuracy_score(y_test, y_pred_random))


Precisión del modelo de Random Search: 0.9101723719349356


In [37]:
import pickle

# Guardar el modelo
filename = 'firstmodel_model.sav'
with open(filename, 'wb') as file:
    pickle.dump(model, file)

print("Modelo guardado exitosamente en", filename)


Modelo guardado exitosamente en firstmodel_model.sav
